# <span style="color:darkblue"> Lecture 15 - Merging Data </span>

<font size = "5">

In the previous class we covered ...

- Aggregate Statistics
- Merge aggregate stats

In this class we will cover ...

- More database merging!
- Emphasize importance of cleaning before merging
- Database concatenation

# <span style="color:darkblue"> I. Import Libraries and Data </span>


<font size = "5">
Key libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


<font size = "5">

Read dataset on car racing circuits

- https://en.wikipedia.org/wiki/Formula_One <br>
- [See Data Source](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020)

In [2]:
results_raw  = pd.read_csv("data_raw/results.csv")
races_raw    = pd.read_csv("data_raw/races.csv")
circuits_raw = pd.read_csv("data_raw/circuits.csv")

<font size = "5">

Multi-file datasets can be visualized with an ...

- "Entity Relationship Diagram" (ERD)
- How the identifiers in each table are connected
- Complement to the "codebook"

<img src="figures/erd_f1_simple.png" alt="drawing" width="600"/>


<font size = "5">

Start by opening datasets!

- Check columns with similar names

In [3]:
# We extract all the unique values in races_raw["name"] and circuits_raw["name"]
# We use "sort_values()" to make it easier to compare the variables
# The "codebook/f1_codebook.pdf" file shows that the content is indeed different

unique_data_races = pd.unique(races_raw["name"].sort_values())
unique_data_circuits = pd.unique(circuits_raw["name"].sort_values())


In [5]:
unique_data_races

array(['70th Anniversary Grand Prix', 'Abu Dhabi Grand Prix',
       'Argentine Grand Prix', 'Australian Grand Prix',
       'Austrian Grand Prix', 'Azerbaijan Grand Prix',
       'Bahrain Grand Prix', 'Belgian Grand Prix', 'Brazilian Grand Prix',
       'British Grand Prix', 'Caesars Palace Grand Prix',
       'Canadian Grand Prix', 'Chinese Grand Prix', 'Dallas Grand Prix',
       'Detroit Grand Prix', 'Dutch Grand Prix', 'Eifel Grand Prix',
       'Emilia Romagna Grand Prix', 'European Grand Prix',
       'French Grand Prix', 'German Grand Prix', 'Hungarian Grand Prix',
       'Indian Grand Prix', 'Indianapolis 500', 'Italian Grand Prix',
       'Japanese Grand Prix', 'Korean Grand Prix', 'Las Vegas Grand Prix',
       'Luxembourg Grand Prix', 'Malaysian Grand Prix',
       'Mexican Grand Prix', 'Mexico City Grand Prix', 'Miami Grand Prix',
       'Monaco Grand Prix', 'Moroccan Grand Prix', 'Pacific Grand Prix',
       'Pescara Grand Prix', 'Portuguese Grand Prix', 'Qatar Grand Prix

In [10]:
unique_data_circuits

array(['AVUS', 'Adelaide Street Circuit', 'Ain Diab', 'Aintree',
       'Albert Park Grand Prix Circuit', 'Autodromo Enzo e Dino Ferrari',
       'Autodromo Internazionale del Mugello',
       'Autodromo Nazionale di Monza', 'Autódromo Hermanos Rodríguez',
       'Autódromo Internacional Nelson Piquet',
       'Autódromo Internacional do Algarve', 'Autódromo José Carlos Pace',
       'Autódromo Juan y Oscar Gálvez', 'Autódromo do Estoril',
       'Bahrain International Circuit', 'Baku City Circuit',
       'Brands Hatch', 'Buddh International Circuit', 'Charade Circuit',
       'Circuit Bremgarten', 'Circuit Gilles Villeneuve',
       'Circuit Mont-Tremblant', 'Circuit Park Zandvoort',
       'Circuit Paul Ricard', 'Circuit de Barcelona-Catalunya',
       'Circuit de Monaco', 'Circuit de Nevers Magny-Cours',
       'Circuit de Pedralbes', 'Circuit de Spa-Francorchamps',
       'Circuit of the Americas', 'Circuito da Boavista',
       'Circuito de Jerez', 'Detroit Street Circuit', 'Dijo

# <span style="color:darkblue"> II. Dictionaries + Renaming </span>

<font size = "5">

A dictionary is another way to store data. 

- Defined with curly brackets "{...}"
- Different fields are separated by a comma
- Assign values to a field with a colon ":"

<font size = "5">

Dictionaries + Pandas

In [4]:
car_dictionary = {"car_model": ["Ferrari","Tesla","BMW"],
                  "year"     : ["2018","2023","2022"]}

In [7]:
matrix_dict = {'A': np.array([[1,2,3], [2,4,5]]), 'string': 'ABC'}

matrix_dict

{'A': array([[1, 2, 3],
        [2, 4, 5]]),
 'string': 'ABC'}

In [8]:
car_dictionary['car_model']

['Ferrari', 'Tesla', 'BMW']

In [9]:
# This is an example of a pandas data frame created from a dictionary
# This example illustrates the basic syntax of a dictionary

car_dictionary = {"car_model": ["Ferrari","Tesla","BMW","Something"],
                  "year": ["2018","2023","2022", "1993"]}

pd.DataFrame(car_dictionary)

,car_model,year
0,Ferrari,2018
1,Tesla,2023
2,BMW,2022
3,Something,1993


<font size = "5">

Rename columns with dictionaries

``` {"old_name": "new_name"} ```

In [12]:
circuits_raw.rename(columns= { 'name' : 'circuit_name'} )

,circuitId,circuitRef,circuit_name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park
...,...,...,...,...,...,...,...,...,...
72,75,portimao,Autódromo Internacional do Algarve,Portimão,Portugal,37.22700,-8.62670,108,http://en.wikipedia.org/wiki/Algarve_Internati...
73,76,mugello,Autodromo Internazionale del Mugello,Mugello,Italy,43.99750,11.37190,255,http://en.wikipedia.org/wiki/Mugello_Circuit
74,77,jeddah,Jeddah Corniche Circuit,Jeddah,Saudi Arabia,21.63190,39.10440,15,http://en.wikipedia.org/wiki/Jeddah_Street_Cir...
75,78,losail,Losail International Circuit,Al Daayen,Qatar,25.49000,51.45420,\N,http://en.wikipedia.org/wiki/Losail_Internatio...


In [13]:
# We first define the dictionary
# Change the pipe ".rename(...)" to rename the columns
# Dictionaries can flexibly accommodate single values or list after ":"

dict_rename_circuits = {"name": "circuit_name"}
circuits = circuits_raw.rename(columns = dict_rename_circuits)

<font size = "5">
Check that ".rename()" worked

In [14]:
# Extract the column names of the "raw" and "clean" data

print("Old List:")
print(circuits_raw.columns.values)
print("")
print("New List:")
print(circuits.columns.values)


Old List:
['circuitId' 'circuitRef' 'name' 'location' 'country' 'lat' 'lng' 'alt'
 'url']

New List:
['circuitId' 'circuitRef' 'circuit_name' 'location' 'country' 'lat' 'lng'
 'alt' 'url']


<font size = 5>

Try it yourself!

- Create a dictionary to rename "name" to "race_name"
- Rename this column in the "races_raw" dataset
- Store the output in a new dataset called "races"

In [ ]:
# Write your own code


dict_rename_race = {"name": "race_name"}
races = races_raw.rename(columns = dict_rename_race)

races


,raceId,year,round,circuitId,race_name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,1116,2023,19,69,United States Grand Prix,2023-10-22,19:00:00,https://en.wikipedia.org/wiki/2023_United_Stat...,2023-10-20,17:30:00,2023-10-21,18:00:00,\N,\N,2023-10-20,21:00:00,2023-10-21,22:00:00
1098,1117,2023,20,32,Mexico City Grand Prix,2023-10-29,20:00:00,https://en.wikipedia.org/wiki/2023_Mexico_City...,2023-10-27,18:30:00,2023-10-27,22:00:00,2023-10-28,17:30:00,2023-10-28,21:00:00,\N,\N
1099,1118,2023,21,18,São Paulo Grand Prix,2023-11-05,17:00:00,https://en.wikipedia.org/wiki/2023_S%C3%A3o_Pa...,2023-11-03,14:30:00,2023-11-04,14:30:00,\N,\N,2023-11-03,18:00:00,2023-11-04,18:30:00
1100,1119,2023,22,80,Las Vegas Grand Prix,2023-11-19,06:00:00,https://en.wikipedia.org/wiki/2023_Las_Vegas_G...,2023-11-17,04:30:00,2023-11-17,08:00:00,2023-11-18,04:30:00,2023-11-18,08:00:00,\N,\N


# <span style="color:darkblue"> II. Merging </span>


<font size = "5">

Extracting specific columns from dataset

In [17]:
circuits[ ["circuitId","circuit_name"] ]

,circuitId,circuit_name
0,1,Albert Park Grand Prix Circuit
1,2,Sepang International Circuit
2,3,Bahrain International Circuit
3,4,Circuit de Barcelona-Catalunya
4,5,Istanbul Park
...,...,...
72,75,Autódromo Internacional do Algarve
73,76,Autodromo Internazionale del Mugello
74,77,Jeddah Corniche Circuit
75,78,Losail International Circuit


<font size = "5">

Merge datasets

<img src="figures/merge_goal.png" alt="drawing" width="500"/>


```pd.merge(data1,data2,on,how)```

- Strive to merge only specific columns of data2
- Avoid merging all columns
- Keeping it simple gives you more control over the output

In [43]:
circuits

,circuitId,circuitRef,circuit_name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park
...,...,...,...,...,...,...,...,...,...
72,75,portimao,Autódromo Internacional do Algarve,Portimão,Portugal,37.22700,-8.62670,108,http://en.wikipedia.org/wiki/Algarve_Internati...
73,76,mugello,Autodromo Internazionale del Mugello,Mugello,Italy,43.99750,11.37190,255,http://en.wikipedia.org/wiki/Mugello_Circuit
74,77,jeddah,Jeddah Corniche Circuit,Jeddah,Saudi Arabia,21.63190,39.10440,15,http://en.wikipedia.org/wiki/Jeddah_Street_Cir...
75,78,losail,Losail International Circuit,Al Daayen,Qatar,25.49000,51.45420,\N,http://en.wikipedia.org/wiki/Losail_Internatio...


In [47]:
circuits[["circuit_name", "location"]]

,circuit_name,location
0,Albert Park Grand Prix Circuit,Melbourne
1,Sepang International Circuit,Kuala Lumpur
2,Bahrain International Circuit,Sakhir
3,Circuit de Barcelona-Catalunya,Montmeló
4,Istanbul Park,Istanbul
...,...,...
72,Autódromo Internacional do Algarve,Portimão
73,Autodromo Internazionale del Mugello,Mugello
74,Jeddah Corniche Circuit,Jeddah
75,Losail International Circuit,Al Daayen


In [18]:
# The "pd.merge()" command combines the information from both datasets
# The first argument is the "primary" datasets
# The second argument is the "secondary" dataset (much include the "on" column)
# The "on" is the common variable that is used for merging
# how = "left" tells Python that the left dataset is the primary one

races_merge = pd.merge(races_raw[['raceId', 'year', 'circuitId']],
                       circuits[["circuitId","circuit_name", "location"]],
                       on = "circuitId",
                       how = "left")

In [23]:
races_merge

,raceId,year,circuitId,circuit_name,location
0,1,2009,1,Albert Park Grand Prix Circuit,Melbourne
1,2,2009,2,Sepang International Circuit,Kuala Lumpur
2,3,2009,17,Shanghai International Circuit,Shanghai
3,4,2009,3,Bahrain International Circuit,Sakhir
4,5,2009,4,Circuit de Barcelona-Catalunya,Montmeló
...,...,...,...,...,...
1097,1116,2023,69,Circuit of the Americas,Austin
1098,1117,2023,32,Autódromo Hermanos Rodríguez,Mexico City
1099,1118,2023,18,Autódromo José Carlos Pace,São Paulo
1100,1119,2023,80,Las Vegas Strip Street Circuit,Las Vegas


In [26]:
races_merge[["raceId",  "circuitId","circuit_name"]].sort_values(by = "circuit_name")

,raceId,circuitId,circuit_name
760,761,61,AVUS
434,435,29,Adelaide Street Circuit
335,336,29,Adelaide Street Circuit
319,320,29,Adelaide Street Circuit
370,371,29,Adelaide Street Circuit
...,...,...,...
501,502,40,Zolder
486,487,40,Zolder
470,471,40,Zolder
516,517,40,Zolder


In [19]:
# Another example of merging

results_merge = pd.merge(results_raw,
                         races_raw[["raceId","date"]],
                         on = "raceId",
                         how = "left")

<font size = "5">
<span style="color:red"> Common pitfall: </span> What happens if you don't rename?

In [20]:
circuits_raw[["circuitId","name"]]

,circuitId,name
0,1,Albert Park Grand Prix Circuit
1,2,Sepang International Circuit
2,3,Bahrain International Circuit
3,4,Circuit de Barcelona-Catalunya
4,5,Istanbul Park
...,...,...
72,75,Autódromo Internacional do Algarve
73,76,Autodromo Internazionale del Mugello
74,77,Jeddah Corniche Circuit
75,78,Losail International Circuit


In [30]:
# The following code merges the raw data
# which has the "name" column in "races_raw" and "circuits_raw"

races_merge_pitfall = pd.merge(races_raw,
                               circuits_raw[["circuitId","name"]],
                               on = "circuitId",
                               how = "left")

# Python will internally rename the columns "name_x" (for the left dataset)
# and "name_y" (for the right dataset)

print(races_merge_pitfall.columns.values)


['raceId' 'year' 'round' 'circuitId' 'name_x' 'date' 'time' 'url'
 'fp1_date' 'fp1_time' 'fp2_date' 'fp2_time' 'fp3_date' 'fp3_time'
 'quali_date' 'quali_time' 'sprint_date' 'sprint_time' 'name_y']


<font size = "5">

Try it yourself!

- Rename the columns "name_x" and "name_y" <br>
in the dataset "races_merge_pitfall" to <br>
 "race_name" and "circuit_name"

$\quad$ HINT: Create a dictionary and use ".rename()"

In [ ]:
# Write your own code




,raceId,year,round,circuitId,name_circuits,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time,name_races
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,Albert Park Grand Prix Circuit
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,Sepang International Circuit
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,Shanghai International Circuit
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,Bahrain International Circuit
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,Circuit de Barcelona-Catalunya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,1116,2023,19,69,United States Grand Prix,2023-10-22,19:00:00,https://en.wikipedia.org/wiki/2023_United_Stat...,2023-10-20,17:30:00,2023-10-21,18:00:00,\N,\N,2023-10-20,21:00:00,2023-10-21,22:00:00,Circuit of the Americas
1098,1117,2023,20,32,Mexico City Grand Prix,2023-10-29,20:00:00,https://en.wikipedia.org/wiki/2023_Mexico_City...,2023-10-27,18:30:00,2023-10-27,22:00:00,2023-10-28,17:30:00,2023-10-28,21:00:00,\N,\N,Autódromo Hermanos Rodríguez
1099,1118,2023,21,18,São Paulo Grand Prix,2023-11-05,17:00:00,https://en.wikipedia.org/wiki/2023_S%C3%A3o_Pa...,2023-11-03,14:30:00,2023-11-04,14:30:00,\N,\N,2023-11-03,18:00:00,2023-11-04,18:30:00,Autódromo José Carlos Pace
1100,1119,2023,22,80,Las Vegas Grand Prix,2023-11-19,06:00:00,https://en.wikipedia.org/wiki/2023_Las_Vegas_G...,2023-11-17,04:30:00,2023-11-17,08:00:00,2023-11-18,04:30:00,2023-11-18,08:00:00,\N,\N,Las Vegas Strip Street Circuit


<font size = "5">

Try it yourself!

- Merge the column "alt", "lng", and "lat" into the races data <br>
using "pd.merge()

In [ ]:
# Write your own code





,raceId,year,round,circuitId,name_x,date,time,url,fp1_date,fp1_time,...,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time,name_y,alt,lng,lat
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,...,\N,\N,\N,\N,\N,\N,Albert Park Grand Prix Circuit,10,144.96800,-37.84970
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,...,\N,\N,\N,\N,\N,\N,Sepang International Circuit,18,101.73800,2.76083
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,...,\N,\N,\N,\N,\N,\N,Shanghai International Circuit,5,121.22000,31.33890
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,...,\N,\N,\N,\N,\N,\N,Bahrain International Circuit,7,50.51060,26.03250
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,...,\N,\N,\N,\N,\N,\N,Circuit de Barcelona-Catalunya,109,2.26111,41.57000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,1116,2023,19,69,United States Grand Prix,2023-10-22,19:00:00,https://en.wikipedia.org/wiki/2023_United_Stat...,2023-10-20,17:30:00,...,\N,\N,2023-10-20,21:00:00,2023-10-21,22:00:00,Circuit of the Americas,161,-97.64110,30.13280
1098,1117,2023,20,32,Mexico City Grand Prix,2023-10-29,20:00:00,https://en.wikipedia.org/wiki/2023_Mexico_City...,2023-10-27,18:30:00,...,2023-10-28,17:30:00,2023-10-28,21:00:00,\N,\N,Autódromo Hermanos Rodríguez,2227,-99.09070,19.40420
1099,1118,2023,21,18,São Paulo Grand Prix,2023-11-05,17:00:00,https://en.wikipedia.org/wiki/2023_S%C3%A3o_Pa...,2023-11-03,14:30:00,...,\N,\N,2023-11-03,18:00:00,2023-11-04,18:30:00,Autódromo José Carlos Pace,785,-46.69970,-23.70360
1100,1119,2023,22,80,Las Vegas Grand Prix,2023-11-19,06:00:00,https://en.wikipedia.org/wiki/2023_Las_Vegas_G...,2023-11-17,04:30:00,...,2023-11-18,04:30:00,2023-11-18,08:00:00,\N,\N,Las Vegas Strip Street Circuit,\N,-115.17300,36.11470
